In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200618']:
    print('----------------------------------------------------------------')
    print(y)
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_52_03_day_pcap\\mdL2Pcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSH1["StockID"] = logSH1["ID"] - 1000000
    logSH1 = logSH1[["sequenceNo", "StockID", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q", 
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", 
                     "ask2q", "ask3q", "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSH1 = logSH1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
        
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SH_*'))[0]
    path2 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SZ_*'))[0]
    path3 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zt_96_04_day_96data\\mdLog_SH_*'))[0]
    
    
    startTm = datetime.datetime.now()
    logSH2 = pd.read_csv(path3,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    print(datetime.datetime.now() - startTm)
    logSH2["time"] = logSH2["time"].apply(lambda x: int((x.replace(':', "")).replace(".", ""))) 
    
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000) \
              & (logSH2["source"] == 13)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "time", "numTrades"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
    
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    
    for cols in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p","ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
    
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        display(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        display(len(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        display(test[np.isnan(test["sequenceNo_y"])]["time"].unique())
        display(len(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
        display(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)
    del logSH2
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    print('----------------------------------------------------------------')
    print('SH index data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_52_03_day_pcap\\mdIndexPcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    index = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    index["StockID"] = index["ID"] - 1000000
    index = index.rename(columns={"open":"openPrice"})

    startTm = datetime.datetime.now()
    logSH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
    in_dex = [16, 300, 852, 905]
    index = index[index["StockID"].isin(in_dex)]
    print(index["StockID"].unique())
    
    data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["time"] >= 91500000) & (logSH["time"] <= 150000000)]
    data2 = index[(index["time"] >= 91500000) & (index["time"] <= 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    
    for cols in ["cum_amount", "close", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
        
        
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice", "time"]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo_x"])])
    
    del index
    del logSH
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    
    print('----------------------------------------------------------------')
    print('SZ lv2 data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdL2Pcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSZ1 = logSZ1.loc[:, ["clockAtArrival", "sequenceNo", "ID", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSZ1 = logSZ1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
    logSZ1["StockID"] = logSZ1["ID"] - 2000000

    startTm = datetime.datetime.now()
    logSZ = pd.read_csv(path2,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    data1 = logSZ[(logSZ["time"] >= 91500000) & (logSZ["time"] < 150000000) & (logSZ["source"] == 4)]
    data2 = logSZ1[(logSZ1["time"] >= 91500000) & (logSZ1["time"] < 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades", "time"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
   
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for i in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[i] = (data1_1[i] * 10000).round(0)

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    print(datetime.datetime.now() - startTm)
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        print(np.sort(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        print(len(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())))
        print(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
    del logSZ
    del logSZ1
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdOrderPcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    OrderLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    OrderLogSZ1["SecurityID"] = OrderLogSZ1["ID"] - 2000000
    OrderLogSZ1 = OrderLogSZ1.rename(columns={"time":'TransactTime'})
    

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdOrderLog_*'))[0]
    OrderLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    
    OrderLogSZ1["OrderType"] = np.where(OrderLogSZ1["OrderType"] == 2, '2', np.where(
        OrderLogSZ1["OrderType"] == 1, '1', OrderLogSZ1['OrderType']))
    

    display(len(OrderLogSZ["SecurityID"].unique()))
    display(len(OrderLogSZ1["SecurityID"].unique()))
    display(len(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique())))
    display(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique()))


    sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
    OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
    OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    
    print('----------------------------------------------------------------')
    print('SZ order data:')
    
    columns = ["ApplSeqNum", "TransactTime", "Side", 'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(ree[np.isnan(ree["sequenceNo_y"])])
        print(len(ree[np.isnan(ree["sequenceNo_y"])]))
        print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("test is complete, baseline is not complete:")
        display(ree[np.isnan(ree["sequenceNo_x"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    del OrderLogSZ
    del OrderLogSZ1
    del ree
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_52_03_day_pcap\\mdTradePcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    SH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    SH1["SecurityID"] = SH1["ID"] - 1000000
    SH1 = SH1.rename(columns={"time":'TransactTime'})

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]
    SH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    SH = SH[SH["exchId"] == 1]
    
    display(len(SH["SecurityID"].unique()))
    display(len(SH1["SecurityID"].unique()))
    display(len(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique())))
    display(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))

    
    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)
    
    print('----------------------------------------------------------------')
    print('SH trade data:')
    
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = 'F'
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    del SH
    del SH1
    del re
    
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdTradePcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    TradeLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    TradeLogSZ1["SecurityID"] = TradeLogSZ1["ID"] - 2000000
    TradeLogSZ1 = TradeLogSZ1.rename(columns={"time":'TransactTime'})
    TradeLogSZ1["TradeBSFlag"] = 'N'
    
    startTm = datetime.datetime.now()
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]

    TradeLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    TradeLogSZ = TradeLogSZ[TradeLogSZ["exchId"] != 1]
    print(datetime.datetime.now() - startTm)
    
    
    display(len(TradeLogSZ["SecurityID"].unique()))
    display(len(TradeLogSZ1["SecurityID"].unique()))
    display(len(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique())))
    display(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique()))


    sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
    TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
    TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))

    print(TradeLogSZ1.columns)
    
    
    print('----------------------------------------------------------------')
    print('SZ trade data:')
    
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo_x"].count()
    n2 = re["sequenceNo_y"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo_y"])])
        print(len(re[np.isnan(re["sequenceNo_y"])]))
        print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo_x"])])
        print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
        display(n2-n1)
    del TradeLogSZ
    del TradeLogSZ1
    del re

----------------------------------------------------------------
20200618


IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
readPath = r'\\mentos\dailyRawData\logs_20200618_zt_52_03_day_pcap\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs

array([], dtype=float64)